# 정확도

: 직관적으로 모델 예측 성능을 나타내는 평가지표
   - 이진분류에서 데이터 구성에 따라 성능을 왜곡할 수 있기에 정확도 하나만 사용 x
   - 불균형한 레이블 값 분포에서 모델 성능을 평가하기에 적합하지 않음
       - ex)레이블 테스트 데이터셋 값이 90%가 1이면 다 1로 예측해도 정확도 0.9 나옴

### 타이타닉 생존자 예측에서 여성은 모두 생존으로 판별

- Dummy Estimator 만들기
    - 남자면 사망, 여자면 생존으로 예측

In [8]:
import numpy as np
from sklearn.base import BaseEstimator

class MyDummyEstimator(BaseEstimator):
    def fit(self, X, y):
        pass
    # 남자면 사망, 여자면 생존으로 예측
    def predict(self, X):
        pred = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i]==1:
                pred[i] = 0
            else:
                pred[i]=1
        return pred

- 데이터 전처리 함수들

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# NULL 처리 함수
def fillna(df):
    df["Age"].fillna(df["Age"].mean(), inplace=True)
    df["Cabin"].fillna('N', inplace=True)
    df["Embarked"].fillna('N', inplace=True)
    df["Fare"].fillna(0, inplace=True)
    return df

# 불필요한 칼럼 삭제
def drop_features(df):
    df.drop(["Name","Ticket","PassengerId"], axis=1, inplace=True)
    return df

# 레이블 인코딩 
def format_features(df):
    df["Cabin"] = df["Cabin"].str[:1]
    features = ["Sex","Cabin","Embarked"]
    for feature in features:
        encoder = LabelEncoder()
        encoder.fit(df[feature])
        df[feature] = encoder.transform(df[feature])
    return df

# 위 세 함수 한꺼번에 실행
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

- Dummy Estimator로 학습/예측/평가

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

titanic_df = pd.read_csv("titanic_train.csv")
X_titanic_df = titanic_df.drop("Survived",axis=1)
X_titanic_df = transform_features(X_titanic_df)
y_titanic_df = titanic_df["Survived"]

X_train,X_test,y_train,y_test = train_test_split(X_titanic_df, y_titanic_df, 
                                                test_size=0.2,random_state=49)

myclf = MyDummyEstimator()
myclf.fit(X_train,y_train)
pred = myclf.predict(X_test)
print("Dummy Classifier의 정확도는 {0:.4f}".format(accuracy_score(pred, y_test)))

Dummy Classifier의 정확도는 0.7765


### MNIST 데이터를 multi classification에서 이진분류로 변경

- Fake Classifier 만들기
    - 모두 0으로 예측

In [11]:
import numpy as np
from sklearn.base import BaseEstimator

class MyFakeClassifier(BaseEstimator):
    def fit(self, X,y):
        pass
    # 모두 0으로 예측
    def predict(self, X):
        return np.zeros(X.shape[0], dtype=bool)

- feature, label 데이터셋 추출

In [14]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

digits = load_digits()
print(digits.data)
print("digits.data의 shape : ",digits.data.shape)
print(digits.target)
print("digits.target의 shape : ",digits.target.shape)

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
digits.data의 shape :  (1797, 64)
[0 1 2 ... 8 9 8]
digits.target의 shape :  (1797,)


1797개의 그림들을 8 * 8 해서 64개의 픽셀로 나눴구나 

In [15]:
digits.target==7

array([False, False, False, ..., False, False, False])

- Fake Classifier로 학습/예측/평가

In [20]:
y = (digits.target==7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state = 49)

print("레이블 테스트 세트 크기 :",y_test.shape)
print("레이브 테스트 세트 0과 1의 분포도")
print(pd.Series(y_test).value_counts())

clf = MyFakeClassifier()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print("모든 예측을 0으로 하여도 정확도는 {0:.4f}".format(accuracy_score(pred,y_test)))

레이블 테스트 세트 크기 : (450,)
레이브 테스트 세트 0과 1의 분포도
0    410
1     40
dtype: int64
모든 예측을 0으로 하여도 정확도는 0.9111
